<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C1/W2/ungraded_labs/C1_W2_Lab_1_beyond_hello_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Beyond Hello World, A Computer Vision Example
In the previous exercise, you saw how to create a neural network that figured out the problem you were trying to solve. This gave an explicit example of learned behavior. Of course, in that instance, it was a bit of overkill because it would have been easier to write the function `y=2x-1` directly instead of bothering with using machine learning to learn the relationship between `x` and `y`.

But what about a scenario where writing rules like that is much more difficult -- for example a computer vision problem? Let's take a look at a scenario where you will build a neural network to recognize different items of clothing, trained from a dataset containing 10 different types.

## Start Coding

Let's start with our import of TensorFlow.

In [ ]:
import tensorflow as tf

print(tf.__version__)

The [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) is a collection of grayscale 28x28 pixel clothing images. Each image is associated with a label as shown in this table⁉

| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

This dataset is available directly in the [tf.keras.datasets](https://www.tensorflow.org/api_docs/python/tf/keras/datasets) API and you load it like this:

In [ ]:
# Load the Fashion MNIST dataset
# create object fmnist from datasets
fmnist = tf.keras.datasets.fashion_mnist

In [ ]:
#fmnist.shape #will error, cause fmnist is an object, not variable.
#need to load dataset to make a variable

Calling `load_data()` on this object will give you two tuples with two lists each. These will be the training and testing values for the graphics that contain the clothing items and their labels.


In [ ]:
# Load the training and test split of the Fashion MNIST dataset
(training_images, training_labels), (test_images, test_labels) = fmnist.load_data()

In [ ]:
training_images.shape

In [ ]:
training_labels.shape

In [ ]:
test_images.shape

In [ ]:
test_labels.shape

In [ ]:
# if want to make as 2-d
xtrain = training_images.reshape(training_images.shape[0],(training_images.shape[1] * training_images.shape[2]))

In [ ]:
xtrain.shape
#from (60000, 28, 28) to (60000, 28x28 = 784)

What do these values look like? Let's print a training image (both as an image and a numpy array), and a training label to see. Experiment with different indices in the array. For example, also take a look at index `42`. That's a different boot than the one at index `0`.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# You can put between 0 to 59999 here
index = 11

# Set number of characters per row when printing
#np.set_printoptions(linewidth=320)

# Print the label and image
print(f'LABEL: {test_labels[index]}') #based on test labels
print(f'\nIMAGE PIXEL ARRAY:\n {test_images[index]}') #based on test images

# Visualize the image
plt.imshow(test_images[index])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# You can put between 0 to 59999 here
index = 11

# Set number of characters per row when printing
#np.set_printoptions(linewidth=320)

# Print the label and image
print(f'LABEL: {training_labels[index]}') #based on labels
print(f'\nIMAGE PIXEL ARRAY:\n {training_images[index]}') #based on training

# Visualize the image
plt.imshow(training_images[index])

You'll notice that all of the values in the number are between 0 and 255. If you are training a neural network especially in image processing, for various reasons it will usually learn better ***if you scale all values to between 0 and 1.*** It's a process called _normalization_ and fortunately in Python, it's easy to normalize an array without looping. You do it like this:

In [ ]:
# Normalize the pixel values of the train and test images
training_images  = training_images / 255.0
test_images = test_images / 255.0

In [ ]:
training_images[15] #normalized

Now you might be wondering why the dataset is split into two: training and testing? Remember we spoke about this in the intro? The idea is to have 1 set of data for training, and then another set of data that the model hasn't yet seen. This will be used to evaluate how good it would be at classifying values.

Let's now design the model. There's quite a few new concepts here. But don't worry, you'll get the hang of them.

In [ ]:
# Build the classification model
#design our model
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),  #input of layer
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)]) #oputput

[Sequential](https://keras.io/api/models/sequential/): That defines a sequence of layers in the neural network.

[Flatten](https://keras.io/api/layers/reshaping_layers/flatten/): Remember earlier where our images were a 28x28 pixel matrix when you printed them out? Flatten just takes that square and turns it into a 1-dimensional array.

[Dense](https://keras.io/api/layers/core_layers/dense/): Adds a layer of neurons

Each layer of neurons need an [activation function](https://keras.io/api/layers/activations/) to tell them what to do. There are a lot of options, but just use these for now:

[ReLU](https://keras.io/api/layers/activations/#relu-function) effectively means:

```
if x > 0:
  return x

else:
  return 0
```

In other words, it only passes values greater than 0 to the next layer in the network.

[Softmax](https://keras.io/api/layers/activations/#softmax-function) takes a list of values and scales these so the sum of all elements will be equal to 1. When applied to model outputs, you can think of the scaled values as the probability for that class. For example, in your classification model which has 10 units in the output dense layer, having the highest value at `index = 4` means that the model is most confident that the input clothing image is a coat. If it is at index = 5, then it is a sandal, and so forth. See the short code block below which demonstrates these concepts. You can also watch this [lecture](https://www.youtube.com/watch?v=LLux1SW--oM&ab_channel=DeepLearningAI) if you want to know more about the Softmax function and how the values are computed.


In [ ]:
# Declare sample inputs and convert to a tensor
inputs = np.array([[1.0, 3.0, 4.0, 2.0]])
inputs = tf.convert_to_tensor(inputs)
print(f'input to softmax function: {inputs.numpy()}')

# Feed the inputs to a softmax activation function
outputs = tf.keras.activations.softmax(inputs)
print(f'output of softmax function: {outputs.numpy()}') #4 is the biggest one, so softmax will be biggest on 4.0

# Get the sum of all values after the softmax
sum = tf.reduce_sum(outputs)
print(f'sum of outputs: {sum}') #total probability must be 1

# Get the index with highest value
prediction = np.argmax(outputs)
print(f'class with highest probability: {prediction}') #4 is index 2

The next thing to do, now that the model is defined, is to actually build it. You do this by compiling it with an optimizer and loss function as before -- and then you train it by calling `model.fit()` asking it to fit your training data to your training labels. It will figure out the relationship between the training data and its actual labels so in the future if you have inputs that looks like the training data, then it can predict what the label for that input is.

In [ ]:
#Try SGD instead of Adam optimzer
model2 = tf.keras.models.Sequential([tf.keras.layers.Flatten(),  #input of layer
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)]) #oputput

model2.compile(optimizer = tf.optimizers.SGD(), #with Adam optimizer
              loss = 'sparse_categorical_crossentropy', #loss function
              metrics=['accuracy']) #with accuracy as a metrics

model2.fit(training_images, training_labels, epochs=5)

In [ ]:
model.compile(optimizer = tf.optimizers.Adam(), #with Adam optimizer
              loss = 'sparse_categorical_crossentropy', #loss function
              metrics=['accuracy']) #with accuracy as a metrics

model.fit(training_images, training_labels, epochs=5)

#### With same hyperparameter Adam is better than SGD, loss is lower and accuracy are bigger than SGD


Once it's done training -- you should see an accuracy value at the end of the final epoch. It might look something like `0.9098`. This tells you that your neural network is about 91% accurate in classifying the training data. That is, it figured out a pattern match between the image and the labels that worked 91% of the time. Not great, but not bad considering it was only trained for 5 epochs and done quite quickly.

But how would it work with unseen data? That's why we have the test images and labels. We can call [`model.evaluate()`](https://keras.io/api/models/model_training_apis/#evaluate-method) with this test dataset as inputs and it will report back the loss and accuracy of the model. Let's give it a try:

In [ ]:
# Evaluate the model on unseen data
model.evaluate(test_images, test_labels)


**Training With 89 %, but Test with 86 %...**